## 1. Understanding Data

In [4]:
import os
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

### Before Start
본격적으로 시작하기 전에 데이터에 대해서 아주 약간만 깊이 있게 이해해보는 시간을 가져보려고 합니다.

### Q. 디스크의 CSV파일의 용량은 그렇게 높진 않은데 메모리로 읽기만 하면 몇 배로 늘어나는 이유는?
캐글을 하시다 보면 이런 경험이 한번쯤은 다들 있으실 것 같습니다.   
분명히 CSV 파일로는 1GB 보다 아래였는데 판다스로 read를 하면 2~3GB로 늘어나는 경우가 종종 있는데, 이유가 무엇일까요?

In [ ]:
PATH='../input/jigsaw-unintended-bias-in-toxicity-classification/'

In [ ]:
os.listdir(PATH)

In [ ]:
## Toxic competition data load
toxic_train=pd.read_csv(os.path.join(PATH,'train.csv'),
                       #dtype=## dtype을 정해줄 수 있다. 이게 없으면 전부 int64, float64로 읽어서 메모리를 너무 많이 차지하게 된다.
                       )

In [ ]:
toxic_train

In [ ]:
toxic_train.head()

In [ ]:
print(toxic_train.shape)
toxic_train.info()

In [ ]:
toxic_train.describe()## 평균값과 최대 최솟값을 모두 볼 수 있게 해줌

In [ ]:
toxic_train.toxicity_annotator_count.plot(kind='kde')

In [ ]:
toxic_train.female.value_counts()

In [ ]:
toxic_train.loc[toxic_train.female==0.8]

In [ ]:
print(type(toxic_train))
toxic_train.apply(lambda x: x['identity_annotator_count'] + x['toxicity_annotator_count'], axis=1)

In [ ]:
print(type(toxic_train.female))
toxic_train.female.map(lambda x: int(x==0.8)+1)

### Load data

In [5]:
PATH = '../input/kakr-4th-competition'
print(os.listdir('../input'))
train = pd.read_csv('../input/kakr4th/train.csv')
test  = pd.read_csv('../input/kakr4th/test.csv')

### 데이터 확인

* id
* age : 나이
* workclass : 고용 형태
* fnlwgt : 사람 대표성을 나타내는 가중치 (final weight의 약자)
* education : 교육 수준
* education_num : 교육 수준 수치
* marital_status: 결혼 상태
* occupation : 업종
* relationship : 가족 관계
* race : 인종
* sex : 성별
* capital_gain : 양도 소득
* capital_loss : 양도 손실
* hours_per_week : 주당 근무 시간
* native_country : 국적
* income : 수익 (예측해야 하는 값)
    * \>50K : 1
    * <=50K : 0

### Pandas 라이브러리
Pandas 라이브러리에서 많이 쓰이는 것들을 위주로 살펴보도록 하겠습니다.

loc, iloc, info, describe, value_counts, head, tail, sample, map, apply, groupby 등

In [ ]:
train

In [ ]:
train.sample(10) ## 바닐라 데이터 확인하기

In [ ]:
train.describe()  ## object는 안나옴

In [ ]:
train.info()

In [ ]:
print(type(train)) ## <class 'pandas.core.frame.DataFrame'>
print(type(train.native_country)) ## <class 'pandas.core.series.Series'>
#train.native_country.value_counts() ## 시리즈 타입은 value_counts가능
train.capital_gain.describe(percentiles=[.99])
#train.loc[train.capital_gain==99999, 'age'] # capital gain 이 99999인것을 찾고 그것의 칼럼을 보자

In [ ]:
train.shape

In [ ]:
train.apply(lambda x: x['age'] + x['hours_per_week'], axis=1)

In [ ]:
## 판다스 사용 팁
train.describe()

In [ ]:
train.capital_gain < 99999

In [ ]:
train.income=train.income.map(lambda x: int(x=='>50K'))# 자료 변형

In [ ]:
train.apply(lambda x:x['age']+x['hours_per_week'],axis=1)

In [ ]:
train.income = train.income.map(lambda x: int(x=='>50K'))

In [ ]:
train.age.plot(kind='kde')

## 2. Data Preprocessing

### 2.1 결측치 처리

In [ ]:
train.apply(lambda x: "?" in list(x), axis=1)## 물음표가 있는지 없는지

In [ ]:
train[train.apply(lambda x: "?" in list(x), axis=1)] ## 물음표 있는 row만 찾아줌

In [ ]:
train.occupation.value_counts()

In [ ]:
train.workclass.value_counts()

### 결측치 처리
삭제? 대치?

In [ ]:
train[(train['workclass'] == '?')&(train['occupation'] == '?')] ## 둘 다 있는 경우 출력

In [ ]:
train[(train['workclass'] != '?')&(train['occupation'] == '?')] ## 다른 경우 출력

In [ ]:
train.loc[(train.age >= 30)&(train.workclass=='?'), 'workclass'] = 'No'

In [ ]:
train.workclass.value_counts()

In [ ]:
train.native_country.value_counts()

In [ ]:
train.loc[(train.age>=30)&(train.workclass=='?'),'workclass',]

### Null check 관련 팁
데이터를 일일이 다 확인하지 않고 결측치가 있는지 확인하는 방법은?

In [ ]:
train.info() ## null이 있는 데이터는 dtype이 무조건 float이다.

### 2.2 이상치 처리
수치형 데이터에 이상한 값이 없는지 한번 확인해보도록 하겠습니다.

In [ ]:
train.describe()

In [ ]:
sum(train.loc[train.capital_gain==99999,'income']==1)

In [ ]:
train.loc[train.capital_gain < 99999].sort_values('capital_gain', ascending=False)

### log 표현

In [ ]:
train["log_capital_gain"] = train.capital_gain.map(lambda x: np.log(x, where=(x!=0)))

In [ ]:
train[['capital_gain', 'log_capital_gain']].describe()

In [ ]:
train.capital_gain.map(lambda x: np.log(x, where=(x!=0))).describe()

### 2.3 Scaling

Min-max Scaler: 
범위가 정해진 값이 필요할 때  
아웃라이어에 민감함
  
Standard Scaler: 
평균을 0, 표준편차를 1로 맞추어 정규분포의 특성을 가지도록 만듦  
아웃라이어에 영향을 덜 받음

In [21]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

mm_scaler = MinMaxScaler()
st_scaler = StandardScaler()

In [22]:
mm_scaler.fit_transform(train['fnlwgt'].values.reshape(-1,1))

In [23]:
mm_scaler.data_max_## fit을 한 자료가 들어있다.fnlwgt의 max값과 일치한다.따라서 mm_scaler를 가지고 
# transformer함수를 적용하면 이 통계치로 test에 적용된다.

In [24]:
# 통계치에 데이터를 적용해서 통계치를 구한다.
train['MM_fnlwgt'] = mm_scaler.fit_transform(train['fnlwgt'].values.reshape(-1,1))
test['MM_fnlwgt'] = mm_scaler.transform(test['fnlwgt'].values.reshape(-1,1))

# mm_scaler가 age의 통계치로 변환된다.
train['MM_age'] = mm_scaler.fit_transform(train['age'].values.reshape(-1,1))
test['MM_age'] = mm_scaler.transform(test['age'].values.reshape(-1,1))

train['ST_fnlwgt'] = st_scaler.fit_transform(train['fnlwgt'].values.reshape(-1,1))
test['ST_fnlwgt'] = st_scaler.transform(test['fnlwgt'].values.reshape(-1,1))

train['ST_age'] = st_scaler.fit_transform(train['age'].values.reshape(-1,1))
test['ST_age'] = st_scaler.transform(test['age'].values.reshape(-1,1))

In [ ]:
train.describe()[['MM_fnlwgt', 'MM_age']]

In [ ]:
train.describe()[['ST_fnlwgt', 'ST_age']].round(6)

In [ ]:
train.describe()[['MM_fnlwgt', 'ST_fnlwgt']].round(6)

## 3. Feature Engineering

### 3.1 변수의 종류
변수 종류에 따라서 어떤 차이점이 있는지 살펴봅시다.

In [3]:
train

### 3.2 인코딩
알고리즘이 이해하기 어려운 Feature들을 어떻게 처리할 지 알아봅시다

In [6]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [7]:
train

### Onehot encoder

In [8]:
oe = OneHotEncoder()
oe_result = oe.fit_transform(train['workclass'].values.reshape(-1, 1)).toarray()

In [9]:
oe.get_feature_names(['workclass']) # 이걸로 dataframe 만들수있음

In [49]:
oe_result

In [12]:
sub=pd.DataFrame(data=oe_result, columns=oe.get_feature_names(['workclass']))

In [14]:
pd.concat([train, sub], axis=1)

원핫인코딩 일괄적으로 하는 간단한 방법

In [51]:
pd.get_dummies(train).shape

### Label encoder

In [53]:
le = LabelEncoder()  ## 단점: 라벨의 순서를 정할 수 없다. 그리고 느리다.
le_result = le.fit_transform(train['workclass'].values.reshape(-1, 1))
le_result.shape

In [23]:
le.inverse_transform

In [21]:
workclass_to_num = dict(zip(train['workclass'].unique(), [0,1,2,3,4,5,6,7,8,9]))

In [22]:
train['workclass'].map(workclass_to_num)

In [55]:
train['workclass'].unique()

사이킷런 인코딩은 불편한점이 너무 많아서.. 직접 만드는 게 나을 수도 있습니다.

### (Target) Mean Encoding

In [ ]:
train

In [9]:
male_positive = train.loc[train['sex']=='Male', "income"]
female_positive = train.loc[train['sex']=='Female', "income"]

In [12]:
male_positive.value_counts()

In [13]:
(male_positive.value_counts() / male_positive.shape[0])

In [27]:
female_positive.value_counts() / female_positive.shape[0]

### onehot 인코딩 일괄적으로 하는 간단한 방법~

In [28]:
dummied = pd.get_dummies(train)

In [29]:
dummied

### 3.3 PCA

In [26]:
from sklearn.decomposition import PCA
pca = PCA(n_components=60, svd_solver='full') ##107개의 train을 60개로 줄여보자
#svd_solver: result보기 위해 필요

In [15]:
train.shape

In [30]:
dummied = dummied.drop(columns=['id', 'income_<=50K', 'income_>50K'])

In [31]:
dummied

In [32]:
X_train_std = st_scaler.fit_transform(dummied)

In [33]:
X_train_std = st_scaler.fit_transform(dummied)
X_train_pca = pca.fit_transform(X_train_std)

In [34]:
X_train_pca.shape

In [35]:
pca.explained_variance_ # 60개의 고유값 가장 큰 분산을 가진 순으로 정렬되어있음

In [36]:
pca.components_ ## 고유 벡터, 107개의 차원이 60으로 나오는 고유값 변환행렬

In [44]:
X_train_2.shape

In [ ]:
pca.explained_variance_ratio_

In [43]:
X_train_2 = X_train_std - X_train_std.mean(axis=0)

res = np.dot(X_train_2, pca.components_.T)
res

In [45]:
np.allclose(X_train_pca, res)

## 4. Before Modeling
시간이 남는다면..